Libraries

In [1]:
import argparse
import datetime
import time
import logging
import math
import os
import sys
import random
import datasets
import torch
from torch.optim import AdamW
from datasets import load_dataset, load_from_disk, concatenate_datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import transformers
from accelerate import Accelerator, DistributedType
from accelerate.logging import get_logger
from accelerate.utils import set_seed, broadcast_object_list
from transformers import (
    BertConfig,
    BertTokenizerFast,
    BertForPreTraining,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    SchedulerType,
    get_scheduler,
)

from selectionstrategies import SubmodStrategy
from accelerate import InitProcessGroupKwargs
from selectionstrategies.helper_fns import taylor_softmax_v1
import numpy as np
import pickle
import faiss

set_seed(42)

/home/twumasimb/miniconda3/envs/ingenious/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup Parameters for args

In [21]:
class Args:
    def __init__(self):
        self.log_dir = "./path/to/log_dir"  # Update with your path
        self.subset_dir = "./path/to/subset_dir"  # Update with your path
        self.output_dir = "./path/to/output_dir"  # Update with your path
        self.preprocessed = False  # Set to True if preprocessed
        self.load_data_from_disk = None  # Set to True if loading data from disk
        self.data_directory = None  # Update with your path if load_data_from_disk is True
        self.dataset_name = None  # Specify dataset name if needed
        self.dataset_config_name = None  # Specify dataset config name if needed
        self.train_file = None  # Path to your training file if applicable
        self.validation_file = None  # Path to your validation file if applicable
        self.validation_split_percentage = 5  # Update this as per your requirement
        self.pad_to_max_length = False  # Set to True to pad to max_length
        self.model_name_or_path = None  # Update with your model name or path
        self.config_name = None  # Specify config name or path if not the same as model_name
        self.hidden_size = 768
        self.num_hidden_layers = 12
        self.num_attention_heads = 12
        self.intermediate_size = 3072
        self.tokenizer_name = None  # Specify tokenizer name or path if not the same as model_name
        self.vocab_size = 30522
        self.use_slow_tokenizer = False  # Set to True to use a slow tokenizer
        self.per_device_train_batch_size = 8
        self.per_device_eval_batch_size = 8
        self.learning_rate = 1e-4
        self.lr_max_steps = 1000000
        self.weight_decay = 0.01
        self.max_train_steps = 250000
        self.gradient_accumulation_steps = 1
        self.lr_scheduler_type = "linear"  # Adjust as needed, ensure it's a valid choice
        self.num_warmup_steps = 10000
        self.seed = None  # Specify a seed for reproducible training if needed
        self.max_seq_length = 128
        self.line_by_line = False  # Set to True if handling distinct lines as sequences
        self.preprocessing_num_workers = 96
        self.preprocess_batch_size = None  # Specify batch size during preprocessing if needed
        self.overwrite_cache = False  # Set to True to overwrite cached sets
        self.mlm_probability = 0.15
        self.short_seq_prob = 0.1
        self.nsp_probability = 0.5
        self.subset_fraction = 0.25
        self.selection_strategy = 'fl'  # Update with your strategy
        self.optimizer = "LazyGreedy"  # Update with your optimizer
        self.select_every = 25000
        self.partition_strategy = "random"  # Update with your strategy
        self.layer_for_similarity_computation = 9
        self.num_partitions = 5000
        self.parallel_processes = 96
        self.num_warmstart_epochs = 10
        self.checkpointing_steps = None  # Specify if needed
        self.resume_from_checkpoint = None  # Specify path if resuming from checkpoint
        self.temperature = 1.0  # Update if using a different temperature
        self.main_process_port = None  # Update with your port
        self.visible_gpus = '0'  # Update with your GPU(s)
        self.init_split = None  # Specify to use a smaller portion of the data. Example: 20 means 80% of the data is used

# Instantiate the args
args = Args()


now = datetime.datetime.now()
timestamp = now.strftime("%d_%m_%Y_%H:%M:%S")

# The following can be modified as needed based on your requirements
args.preprocessed = True
args.visible_gpus = '1'  # Assuming you want to use the first GPU; adjust as needed
args.main_process_port = 29500  # Example port; adjust as needed
args.log_dir = f"./output/logs/genious{timestamp}/"
args.model_dir = f"./output/models/genious{timestamp}/"
args.subset_dir = f"./output/subsets/genious{timestamp}/"
args.load_data_from_disk = None
args.data_directory = "./data"
args.tokenizer_name = "bert-base-uncased"
args.preprocess_batch_size = 100
args.per_device_train_batch_size = 4
args.per_device_eval_batch_size = 4
args.learning_rate = 1e-4
args.lr_max_steps = 1000
args.weight_decay = 0.01
args.max_train_steps = 10
args.gradient_accumulation_steps = 1
args.num_warmup_steps = 10
args.output_dir = args.model_dir
args.model_type = "bert"
args.max_seq_length = 64
args.preprocessing_num_workers = 10
args.mlm_probability = 0.15
args.short_seq_prob = 0.1
args.nsp_probability = 0.5
args.subset_fraction = 0.25
args.update_losses_every = 1250
args.checkpointing_steps = 250 
args.init_split = 95 # ignore 95% of the data
args.dataset_name = "bookcorpus"


# Set CUDA visible devices
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

debug = True

# Create directories
os.makedirs(args.log_dir, exist_ok=True)
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.subset_dir, exist_ok=True)

Initialize Logger & Accelerator

In [3]:
logger=get_logger(__name__)
init_process_group=InitProcessGroupKwargs(timeout=datetime.timedelta(seconds=75000))
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator=Accelerator(kwargs_handlers=[init_process_group])
# Make one log on every process with the configuration for debugging
logging.basicConfig(
    filename=os.path.join(args.log_dir,"train_logs.log"),
    filemode="w",
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)

# Setup logging, we only want one process per machine to log things on the screen.
# accelerator.is_local_main_process is only True for one process per machine.
# logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

Load dataset

In [4]:
# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

if accelerator.is_main_process:
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()
if args.preprocessed == False:
    logger.info(f"Data is not preprocessed.")
    logger.info(f"Loading the data.")
    if args.load_data_from_disk is not None:
        if args.data_directory is not None:
            raw_datasets=load_from_disk(args.data_directory)
            if "validation" not in raw_datasets.keys():
                raw_datasets=raw_datasets["train"].train_test_split(test_size=(args.validation_split_percentage/100), shuffle=False)
                raw_datasets=datasets.DatasetDict({"train": raw_datasets["train"], "validation": raw_datasets["test"]})
    elif args.dataset_name is not None:
        raw_datasets=load_dataset(args.dataset_name, args.dataset_config_name)
        if "validaton" not in raw_datasets.keys() and args.init_split is not None:
            raw_datasets=raw_datasets["train"].train_test_split(test_size=(args.init_split/100), shuffle=False)
            raw_datasets=raw_datasets["train"].train_test_split(test_size=(args.validation_split_percentage/100), shuffle=False)
            raw_datasets=datasets.DatasetDict({"train": raw_datasets["train"], "validation": raw_datasets["test"]})
        elif "validation" not in raw_datasets.keys():
            raw_datasets=raw_datasets.train_test_split(test_size=(args.validation_split_percentage/100), shuffle=False)
            raw_datasets=datasets.DatasetDict({"train": raw_datasets["train"], "validation": raw_datasets["test"]})
    else:
        data_files={}
        if args.train_file is not None:
            data_files['train']=args.train_file
        if args.validation_file is not None:
            data_files['validation']=args.validation_file
        extension=args.train_file.split(".")[-1]
        if extension=='txt':
            extension='text'
        raw_datasets=load_dataset(extension, data_files=data_files)
        if "validation" not in raw_datasets.keys():
            raw_datasets=raw_datasets["train"].train_test_split(test_size=(args.validation_split_percentage/100), shuffle=False)
            raw_datasets=datasets.DatasetDict({"train": raw_datasets["train"], "validation": raw_datasets["test"]})

Configure Model and Tokenizer

In [5]:
# Load pretrained model and tokenizer
#
# In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
logger.info(f"Loading the model configuration.")
if args.config_name:
    config=BertConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config=BertConfig.from_pretrained(args.model_name_or_path)
else:
    config=BertConfig(
        vocab_size=args.vocab_size,
        hidden_size=args.hidden_size,
        num_hidden_layers=args.num_hidden_layers,
        num_attention_heads=args.num_attention_heads,
        intermediate_size=args.intermediate_size,
        hidden_act="gelu",
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
        max_position_embeddings=512,
        type_vocab_size=2,
        initializer_range=0.02,
        layer_norm_eps=1e-12,
        position_embedding_type="absolute",
    )

logger.info(f"Loading the tokenizer.")
if args.tokenizer_name:
    tokenizer=BertTokenizerFast.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer=BertTokenizerFast.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

logger.info(f"Initializing Model.")
if args.model_name_or_path:
    model=BertForPreTraining.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt"in args.model_name_or_path),
        config=config
    )
else:
    logger.info("Training a new model from scratch")
    model=BertForPreTraining(config)

model.resize_token_embeddings(len(tokenizer))

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/twumasimb/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /home/twumasimb/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /home/twumasimb/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.76ea01b4b85ac16e2cec55c398c

Embedding(30522, 768, padding_idx=0)

# Preprocessing the Data

Tokenizing all the data

In [6]:
#Preprocessing the datasets
#First we tokenize all the texts
if args.preprocessed == False:
    column_names=raw_datasets['train'].column_names
    text_column_name="text" if "text" in column_names else column_names[0]
else:
    column_names=["text"]
    text_column_name="text"

if args.max_seq_length is None:
    max_seq_length=tokenizer.model_max_length
    if max_seq_length>1024:
        logger.warning(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length})."
            "Picking 1024 instead. You can change the default value by passing --max_seq_length xxx"
        )
        max_seq_length=1024
else:
    if args.max_seq_length>tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the "
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )
    max_seq_length=min(args.max_seq_length, tokenizer.model_max_length)

Process the raw data

In [7]:
#Main data processing function that will concatenate all texts from our dataset and generate chunks of 
#max_seq_length.

def group_texts(examples, idx, split, tokenized_datasets):
    # Account for [CLS], [SEP], [SEP]
    max_num_tokens=max_seq_length-3
    # We *usually* want to fill up the entire sequence since we are padding
    # to `max_seq_length` anyways, so short sequences are generally wasted
    # computation. However, we *sometimes*
    # (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
    # sequences to minimize the mismatch between pre-training and fine-tuning.
    # The `target_seq_length` is just a rough target however, whereas
    # `max_seq_length` is a hard limit.
    target_seq_length=max_num_tokens
    if random.random()<args.short_seq_prob:
        target_seq_length=random.randint(2, max_num_tokens)
    # We DON'T just concatenate all of the tokens from a document into a long
    # sequence and choose an arbitrary split point because this would make the
    # next sentence prediction task too easy. Instead, we split the input into
    # segments "A" and "B" based on the actual "sentences" provided by the user
    # input.
    result={k: [] for k, v in tokenizer("", return_special_tokens_mask=True).items()}
    result['next_sentence_label']=[]
    current_chunk=[]
    current_length=0
    i=0 
    while i<len(idx):
        segment={k: examples[k][i][1:-1] for k in examples.keys()}
        current_chunk.append(segment)
        current_length += len(segment['input_ids'])
        if i==len(idx)-1 or current_length>=target_seq_length:
            if current_chunk:
                # `a_end` is how many segments from `current_chunk` go into the `A`
                # (first) sentence.
                a_end=1
                if len(current_chunk)>=2:
                    a_end=random.randint(1, len(current_chunk)-1)
                tokens_a={k: [] for k, t in tokenizer("", return_special_tokens_mask=True).items()}
                for j in range(a_end):
                    for k, v in current_chunk[j].items():
                        tokens_a[k].extend(v)

                tokens_b={k: [] for k, t in tokenizer("", return_special_tokens_mask=True).items()}
                # Random next
                is_random_next=False
                if len(current_chunk)==1 or random.random()<args.nsp_probability:
                    is_random_next=True
                    target_b_length=target_seq_length-len(tokens_a["input_ids"])
                    # This should rarely go for more than one iteration for large
                    # corpora. However, just to be careful, we try to make sure that
                    # the random document is not the same as the document
                    # we're processing.
                    for _ in range(10):
                        random_segment_index=random.randint(0, len(tokenized_datasets[split])-len(idx)-1)
                        if (random_segment_index-len(idx) not in idx) and (random_segment_index+len(idx) not in idx):
                            break

                    random_start=random.randint(0, len(idx)-1)
                    for j in range(random_start, len(idx)):
                        for k, v in {k: tokenized_datasets[split][random_segment_index+j][k][1:-1] for k in examples.keys()}.items():
                            tokens_b[k].extend(v)
                        if len(tokens_b['input_ids'])>=target_b_length:
                            break
                    # We didn't actually use these segments so we "put them back" so
                    # they don't go to waste.
                    num_unused_segments=len(current_chunk)-a_end
                    i-=num_unused_segments
                # Actual next
                else:
                    is_random_next=False
                    for j in range(a_end, len(current_chunk)):
                        for k, v in current_chunk[j].items():
                            tokens_b[k].extend(v)

                while True:
                    total_length=len(tokens_a['input_ids'])+len(tokens_b['input_ids'])
                    if total_length<=max_num_tokens:
                        break
                    trunc_tokens= tokens_a if len(tokens_a['input_ids'])>len(tokens_b['input_ids']) else tokens_b
                    # We want to sometimes truncate from the front and sometimes from the
                    # back to add more randomness and avoid biases.
                    if random.random()<0.5:
                        for k in trunc_tokens.keys():
                            del trunc_tokens[k][0]
                    else:
                        for k in trunc_tokens.keys():
                            trunc_tokens[k].pop()
                inp={k: v[:-1] for k, v in tokenizer("", return_special_tokens_mask=True).items()}
                for k, v in tokens_a.items():
                    inp[k].extend(v)
                SEP={k: v[1:] for k, v in tokenizer("", return_special_tokens_mask=True).items()}
                for k, v in SEP.items():
                    inp[k].extend(v)
                tokens_b['token_type_ids']=list(map(lambda x: 1, tokens_b['token_type_ids']))
                for k, v in SEP.items():
                    tokens_b[k].extend(v)
                tokens_b['token_type_ids'][-1]=1
                for k, v in tokens_b.items():
                    inp[k].extend(v)
                inp['next_sentence_label']=int(is_random_next)
                for k, v in inp.items():
                    result[k].append(v)
            current_chunk=[]
            current_length=0
        i+=1
    return result

In [8]:
if args.preprocessed == False:
    logger.info(f"Beginning Tokenization.")
    if args.line_by_line == True:
        #when using line_by_line, we just tokenize each non-empty line.
        padding="max_length" if args.pad_to_max_length==True else False

        def tokenize_function(examples):
            #remove empty lines
            examples[text_column_name]=[
                line for line in examples[text_column_name] if len(line)>0 and not line.isspace()
            ]
            return tokenizer(
                examples[text_column_name],
                padding=padding,
                truncation=True,
                max_length=max_seq_length,
                #we use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
                #receives the `special_tokens_mask`
                return_special_tokens_mask=True
            )
        
        with accelerator.main_process_first():
            tokenized_datasets=raw_datasets.map(
                tokenize_function,
                batched=True,
                num_proc=args.preprocessing_num_workers,
                remove_columns=[text_column_name],
                load_from_cache_file=not args.overwrite_cache,
                desc="Running tokenizer on dataset line by line",
            )
        train_dataset=tokenized_datasets["train"]
        eval_dataset=tokenized_datasets["validation"]
    else:
        # otherwise, we tokenize every text, then concatenate them together before splitting them in smaller parts.
        # We use `return_special_tokens=True` because DataCollatorForLanguageModeling (see below) is more efficient when it
        # receives the `special_tokens_mask`.
        def tokenize_function(examples):
            return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

        with accelerator.main_process_first():
            tokenized_datasets=raw_datasets.map(
                tokenize_function,
                batched=True,
                num_proc=args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not args.overwrite_cache,
                desc="Running tokenizer on every text in dataset",
            )
        
        # Note that with `batched=True`, this map processes 1000 texts together, so group_texts throws away a 
        # remainder for each of those groups of 1000 texts. You can adjust that batch_size here, but a higher value
        # might be slower to preprocess.
        #
        # To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
        # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
        train_dataset=tokenized_datasets["train"]
        eval_dataset=tokenized_datasets["validation"]
        logger.info(f"Grouping the tokenized dataset into chunks of {max_seq_length}.")
        with accelerator.main_process_first():
            train_dataset=train_dataset.map(
                group_texts,
                fn_kwargs={'split': 'train', 'tokenized_datasets': tokenized_datasets},
                batched=True,
                batch_size=args.preprocess_batch_size,
                num_proc=args.preprocessing_num_workers,
                load_from_cache_file=not args.overwrite_cache,
                with_indices=True,
                desc=f"Grouping Train texts in chunks of {max_seq_length}",
            )
        with accelerator.main_process_first():
            eval_dataset=eval_dataset.map(
                group_texts,
                fn_kwargs={'split': 'validation', 'tokenized_datasets': tokenized_datasets},
                batched=True,
                batch_size=args.preprocess_batch_size,
                num_proc=args.preprocessing_num_workers,
                load_from_cache_file=not args.overwrite_cache,
                with_indices=True,
                desc=f"Grouping Validation texts in chunks of {max_seq_length}",
            )

        prepared_data  = datasets.DatasetDict({"train": train_dataset, "validation": eval_dataset})
        prepared_data.save_to_disk("./data/bookcorpus/bert/prepared")
        
        dataset=prepared_data['train']
        tokenizer=BertTokenizerFast.from_pretrained("bert-base-uncased")

        def extract_first_sentences(examples):
            for i, input_ids in enumerate(examples["input_ids"]):
                idx=input_ids.index(tokenizer.sep_token_id)
                examples["input_ids"][i]=input_ids[:idx+1]
                examples["attention_mask"][i]=examples["attention_mask"][i][:idx+1]
                examples["token_type_ids"][i]=examples["token_type_ids"][i][:idx+1]
                examples["special_tokens_mask"][i]=examples["special_tokens_mask"][i][:idx+1]
            return examples

        # filter points from dataset with next_sentence_label == 0
        with accelerator.main_process_first():
            nsp_zero=dataset.filter(lambda examples: [x==0 for x in examples["next_sentence_label"]], batched=True, num_proc=args.preprocessing_num_workers, keep_in_memory=True)
            nsp_zero.save_to_disk("./data/bookcorpus/bert/nsp_zero")
                
        # filter points from dataset with next_sentence_label == 1
        with accelerator.main_process_first():
                nsp_one=dataset.filter(lambda examples: [x==1 for x in examples["next_sentence_label"]], batched=True, num_proc=args.preprocessing_num_workers, keep_in_memory=True)
                nsp_one.save_to_disk("./data/bookcorpus/bert/nsp_one")
        
        # extract first sentences from both datasets
        with accelerator.main_process_first():
            first_sent_nsp_zero=nsp_zero.map(extract_first_sentences, batched=True, num_proc=args.preprocessing_num_workers, remove_columns=["next_sentence_label", "special_tokens_mask"], keep_in_memory=True)
            first_sent_nsp_zero.save_to_disk("./data/bookcorpus/bert/first_sent_nsp_zero")
        with accelerator.main_process_first():
            first_sent_nsp_one=nsp_one.map(extract_first_sentences, batched=True, num_proc=args.preprocessing_num_workers, remove_columns=["next_sentence_label", "special_tokens_mask"], keep_in_memory=True)
            first_sent_nsp_one.save_to_disk("./data/bookcorpus/bert/first_sent_nsp_one")

If data is already preprocessed, load it them.

In [9]:
if args.preprocessed == True:
    logger.info(f"Loading preprocessed dataset(s) from disk.")
    dataset=load_from_disk("./data/bookcorpus/bert/prepared")
    train_dataset=dataset["train"]
    eval_dataset=dataset["validation"]

    nsp_zero=load_from_disk("./data/bookcorpus/bert/nsp_one")
    nsp_one=load_from_disk("./data/bookcorpus/bert/nsp_zero")
    first_sent_nsp_zero = load_from_disk("./data/bookcorpus/bert/first_sent_nsp_zero")
    first_sent_nsp_one = load_from_disk("./data/bookcorpus/bert/first_sent_nsp_one")

Initializing the random data subset selection process

In [10]:
#Initial Random Subset Selection
if accelerator.is_main_process:
    num_samples = int(round(len(train_dataset) * args.subset_fraction, 0))
    init_subset_indices = [random.sample(list(range(len(train_dataset))), num_samples)]
else:
    init_subset_indices = [[]]
accelerator.wait_for_everyone()
broadcast_object_list(init_subset_indices)
full_dataset=train_dataset
subset_dataset = full_dataset.select(init_subset_indices[0])

logger.info(f"Full data has {len(full_dataset)} samples, subset data has {len(subset_dataset)} samples.")
# Conditional for small test subsets
if len(train_dataset)>3:
    # Log a few random samples from the training data
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

Data Collators and Data Loaders

In [11]:
# Data Collator
# This one will take care of the randomly masking the tokens.
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=args.mlm_probability)
data_collator_embd=DataCollatorWithPadding(tokenizer=tokenizer)
logger.info(f"Creating Data Loaders")
# Dataloaders creation
warmstart_dataloader=DataLoader(
    train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
)

first_sent_nsp_zero_dataloader=DataLoader(
    first_sent_nsp_zero, shuffle=False, collate_fn=data_collator_embd, batch_size=args.per_device_eval_batch_size
)
first_sent_nsp_one_dataloader=DataLoader(
    first_sent_nsp_one, shuffle=False, collate_fn=data_collator_embd, batch_size=args.per_device_eval_batch_size
)

subset_dataloader=DataLoader(
    subset_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
)

eval_dataloader=DataLoader(
    eval_dataset, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
)

Initialize Optimizer 

In [12]:
logger.info(f"Intializing optimizer, learning rate schedule")
# Optimizer
# Split weights in two groups, one with weight decay and the other not
no_decay=["bias", "LayerNorm.weight"]
optimizer_grouped_parameters=[
    {
        "params":[p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay":args.weight_decay,
    },
    {
        "params":[p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0
    }
]

optimizer=AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

Init for TPU & Init Scheduler 

In [13]:
# On TPU, the tie weights in our model have been disconnected, so we need to restore the ties.
if accelerator.distributed_type==DistributedType.TPU:
    model.tie_weights()

lr_scheduler=get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.lr_max_steps,
)

Prepare the model for the accelerator

In [14]:
logger.info(f"Prepare model, optimizer, warmstart_dataloader, first_sent_nsp_zero_dataloader, first_sent_nsp_one_dataloader, subset_dataloader, eval_dataloader with accelerate.")
# Prepare everything with our `accelerator`
model, optimizer, warmstart_dataloader, first_sent_nsp_zero_dataloader, first_sent_nsp_one_dataloader, subset_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, warmstart_dataloader, first_sent_nsp_zero_dataloader, first_sent_nsp_one_dataloader, subset_dataloader, eval_dataloader)

if args.selection_strategy in ['fl', 'logdet', 'gc', 'disparity-sum']:
    subset_strategy = SubmodStrategy(logger, args.selection_strategy,
                                num_partitions=args.num_partitions, partition_strategy=args.partition_strategy,
                                optimizer=args.optimizer, similarity_criterion='feature', 
                                metric='cosine', eta=1, stopIfZeroGain=False, 
                                stopIfNegativeGain=False, verbose=False, lambdaVal=1)
# Figure out how many steps we should save the Accelerator states
if hasattr(args.checkpointing_steps, "isdigit"):
    checkpointing_steps=args.checkpointing_steps
    if args.checkpointing_steps.isdigit():
        checkpointing_steps=int(args.checkpointing_steps)
else:
    checkpointing_steps=None

# Beginning Training

Warmstarting the model

In [15]:
# Train!
print(f"Training on device: {accelerator.device}")
total_batch_size=args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps
main_start_time=time.time()
logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num warm-start epochs = {args.num_warmstart_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
# Potentially load in the weights and states from a previous save
if args.resume_from_checkpoint != None:
    accelerator.print(f"Resumed from checkpoint: {args.resume_from_checkpoint}")
    accelerator.load_state(args.resume_from_checkpoint)

logger.info(f"Begin the training.")
timing = []
warmstart_start_time=time.time()
for epoch in range(args.num_warmstart_epochs):
    if epoch==0:
        logger.info("Begin the warm-start")
    model.train()
    for step, batch in enumerate(warmstart_dataloader):
        start_time=time.time()
        outputs=model(**batch)
        loss=outputs.loss
        logger.info(f"Completed Steps: {1+completed_steps}; Loss: {loss.detach().float()}; lr: {lr_scheduler.get_last_lr()};")
        loss=loss/args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step%args.gradient_accumulation_steps==0 or step==len(warmstart_dataloader)-1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps+=1
        if isinstance(checkpointing_steps, int):
            if completed_steps%checkpointing_steps==0:
                output_dir=f"step_{completed_steps }"
                if args.output_dir is not None:
                    output_dir=os.path.join(args.output_dir, output_dir)
                accelerator.save_state(output_dir)
        if completed_steps>=args.max_train_steps:
            break
        timing.append([(time.time() - start_time), 0])
    
    model.eval()
    losses=[]
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs=model(**batch)

        loss=outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_eval_batch_size)))

    losses=torch.cat(losses)
    losses=losses[:len(eval_dataset)]
    try:
        perplexity=math.exp(torch.mean(losses))
    except OverflowError:
        perplexity=float("inf")

    logger.info(f"Steps {completed_steps}: perplexity: {perplexity}")
    if epoch==args.num_warmstart_epochs-1:
        logger.info("End the warm-start")
# Save the state after warm-start
output_dir=f"after_warmstart_step_{completed_steps}"
if args.output_dir is not None:
    output_dir=os.path.join(args.output_dir, output_dir)
accelerator.save_state(output_dir)
warmstart_end_time=time.time()
logger.info(f"Completed warm-start in {warmstart_end_time-warmstart_start_time} seconds")

19it [31:40, 203.31s/it]                       

Data subselection & Restructuring for model training

In [22]:
probs_nsp_zero=[]
probs_nsp_one=[]
greedyList_nsp_zero=[]
greedyList_nsp_one=[]
gains_nsp_zero=[]
gains_nsp_one=[]

if (args.num_warmstart_epochs!=0) or (args.resume_from_checkpoint != None):
    logger.info(f"Beginning the subset selection after warm-start or resuming from checkpoint")
    start_time=time.time()
    if args.selection_strategy == 'Random-Online':
        if accelerator.is_main_process:
            subset_indices_nsp_zero = [random.sample(list(range(len(first_sent_nsp_zero))), math.floor(num_samples/2))]
            subset_indices_nsp_one = [random.sample(list(range(len(first_sent_nsp_one))), math.ceil(num_samples/2))]
        else:
            subset_indices_nsp_zero = [[]]
            subset_indices_nsp_one = [[]]
    elif args.selection_strategy in ["fl", "logdet", "gc", "disparity-min"]:
        logger.info(f"Performing Subset selection for NSP class 0")
        pbar=tqdm(range(len(first_sent_nsp_zero_dataloader)), disable=not accelerator.is_local_main_process)
        model.eval()
        representations_nsp_zero=[]
        batch_indices_nsp_zero=[]
        total_cnt=0
        total_storage=0

        accelerator.wait_for_everyone()
        unwrapped_model=accelerator.unwrap_model(model)
        bert_model=unwrapped_model.bert
        bert_model=accelerator.prepare(bert_model)
        representations_start_time=time.time()
        for step, batch in enumerate(first_sent_nsp_zero_dataloader):
            with torch.no_grad():
                output=bert_model(**batch, output_hidden_states=True)
            embeddings=output["hidden_states"][args.layer_for_similarity_computation]
            mask=(batch['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float())
            mask1=((batch['token_type_ids'].unsqueeze(-1).expand(embeddings.size()).float())==0)
            mask=mask*mask1
            mean_pooled=torch.sum(embeddings*mask, 1) / torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = accelerator.gather(mean_pooled)
            total_cnt += mean_pooled.size(0)
            if accelerator.is_main_process:
                mean_pooled = mean_pooled.cpu()
                total_storage += sys.getsizeof(mean_pooled.storage())
                representations_nsp_zero.append(mean_pooled)
            pbar.update(1)
        if accelerator.is_main_process:
            representations_nsp_zero=torch.cat(representations_nsp_zero, dim=0)
            representations_nsp_zero=representations_nsp_zero[:len(first_sent_nsp_zero)]
            total_storage += sys.getsizeof(representations_nsp_zero.storage())
            representations_nsp_zero=representations_nsp_zero.numpy()
            logger.info('Representations(NSP Class 0) Size: {}, Total number of samples: {}'.format(total_storage/(1024 * 1024), total_cnt))
            batch_indices_nsp_zero=list(range(len(first_sent_nsp_zero)))
            logger.info('Length of indices: {}'.format(len(batch_indices_nsp_zero)))
            logger.info('Representations(NSP Class 0) gathered. Shape of representations: {}. Length of indices: {}'.format(representations_nsp_zero.shape, len(batch_indices_nsp_zero)))
        logger.info(f"Representations(NSP Class 0) computed in {time.time()-representations_start_time} seconds")
        if accelerator.is_main_process:
            partition_indices_nsp_zero, greedyIdx_nsp_zero, gains_nsp_zero = subset_strategy.select(len(batch_indices_nsp_zero)-1, batch_indices_nsp_zero, representations_nsp_zero, parallel_processes=args.parallel_processes, return_gains=True)
            subset_indices_nsp_zero = [[]]
            i=0
            for p in gains_nsp_zero:
                greedyList_nsp_zero.append(greedyIdx_nsp_zero[i:i+len(p)])         
                i+=len(p)
            probs_nsp_zero=[taylor_softmax_v1(torch.from_numpy(np.array([partition_gains])/args.temperature)).numpy()[0] for partition_gains in gains_nsp_zero]
            rng=np.random.default_rng(args.seed+completed_steps)
            for i, partition_prob in enumerate(probs_nsp_zero):
                partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_zero)) * math.floor(num_samples/2)), len(partition_prob)-1)
                subset_indices_nsp_zero[0].extend(rng.choice(greedyList_nsp_zero[i], size=partition_budget, replace=False, p=partition_prob).tolist())
        else:
            subset_indices_nsp_zero=[[]]
    
        logger.info(f"Performing Subset selection for NSP class 1")
        pbar=tqdm(range(len(first_sent_nsp_one_dataloader)), disable=not accelerator.is_local_main_process)
        model.eval()
        representations_nsp_one=[]
        batch_indices_nsp_one=[]
        total_cnt=0
        total_storage=0
        representations_start_time=time.time()
        for step, batch in enumerate(first_sent_nsp_one_dataloader):
            with torch.no_grad():
                output=bert_model(**batch, output_hidden_states=True)
            embeddings=output["hidden_states"][args.layer_for_similarity_computation]
            mask=(batch['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float())
            mask1=((batch['token_type_ids'].unsqueeze(-1).expand(embeddings.size()).float())==0)
            mask=mask*mask1
            mean_pooled=torch.sum(embeddings*mask, 1) / torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = accelerator.gather(mean_pooled)
            total_cnt += mean_pooled.size(0)
            if accelerator.is_main_process:
                mean_pooled = mean_pooled.cpu()
                total_storage += sys.getsizeof(mean_pooled.storage())
                representations_nsp_one.append(mean_pooled)
            pbar.update(1)
        if accelerator.is_main_process:
            representations_nsp_one=torch.cat(representations_nsp_one, dim=0)
            representations_nsp_one=representations_nsp_one[:len(first_sent_nsp_one)]
            total_storage += sys.getsizeof(representations_nsp_one.storage())
            representations_nsp_one=representations_nsp_one.numpy()
            logger.info('Representations(NSP Class 1) Size: {}, Total number of samples: {}'.format(total_storage/(1024 * 1024), total_cnt))
            batch_indices_nsp_one=list(range(len(first_sent_nsp_one)))
            logger.info('Length of indices: {}'.format(len(batch_indices_nsp_one)))
            logger.info('Representations(NSP Class 1) gathered. Shape of representations: {}. Length of indices: {}'.format(representations_nsp_one.shape, len(batch_indices_nsp_one)))
        logger.info(f"Representations(NSP Class 1) computed in {time.time()-representations_start_time} seconds")
        if accelerator.is_main_process:
            partition_indices_nsp_one, greedyIdx_nsp_one, gains_nsp_one = subset_strategy.select(len(batch_indices_nsp_one)-1, batch_indices_nsp_one, representations_nsp_one, parallel_processes=args.parallel_processes, return_gains=True)
            subset_indices_nsp_one = [[]]
            i=0
            for p in gains_nsp_one:
                greedyList_nsp_one.append(greedyIdx_nsp_one[i:i+len(p)])         
                i+=len(p)
            probs_nsp_one=[taylor_softmax_v1(torch.from_numpy(np.array([partition_gains])/args.temperature)).numpy()[0] for partition_gains in gains_nsp_one]
            rng=np.random.default_rng(args.seed+completed_steps)
            for i, partition_prob in enumerate(probs_nsp_one):
                partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_one)) * math.ceil(num_samples/2)), len(partition_prob)-1)
                subset_indices_nsp_one[0].extend(rng.choice(greedyList_nsp_one[i], size=partition_budget, replace=False, p=partition_prob).tolist())
        else:
            subset_indices_nsp_one=[[]]
    accelerator.wait_for_everyone()    
    broadcast_object_list(subset_indices_nsp_zero)
    broadcast_object_list(subset_indices_nsp_one)
    timing.append([0, time.time()-start_time])
    logger.info(f"First subset selection completed. Total Time taken(including embeddings computation): {time.time()-start_time}")

if accelerator.is_main_process:
    output_file=f"nsp_zero_subset_indices_after_step_{completed_steps}.pt"
    output_file=os.path.join(args.subset_dir, output_file)
    torch.save(torch.tensor(subset_indices_nsp_zero), output_file)
    output_file=f"nsp_one_subset_indices_after_step_{completed_steps}.pt"
    output_file=os.path.join(args.subset_dir, output_file)
    torch.save(torch.tensor(subset_indices_nsp_one), output_file)
    output_file=f"nsp_zero_gains_after_step_{completed_steps}.pkl"
    output_file=os.path.join(args.subset_dir, output_file)
    with open(output_file, "wb") as f:
        pickle.dump(gains_nsp_zero, f)
    output_file=f"nsp_one_gains_after_step_{completed_steps}.pkl"
    output_file=os.path.join(args.subset_dir, output_file)
    with open(output_file, "wb") as f:
        pickle.dump(gains_nsp_one, f)
    output_file=f"nsp_zero_partition_indices_after_step_{completed_steps}.pkl"
    output_file=os.path.join(args.subset_dir, output_file)
    with open(output_file, "wb") as f:
        pickle.dump(partition_indices_nsp_zero, f)
    output_file=f"nsp_one_partition_indices_after_step_{completed_steps}.pkl"
    output_file=os.path.join(args.subset_dir, output_file)
    with open(output_file, "wb") as f:
        pickle.dump(partition_indices_nsp_one, f)
    output_file=f"nsp_zero_greedy_indices_after_step_{completed_steps}.pkl"
    output_file=os.path.join(args.subset_dir, output_file)
    with open(output_file, "wb") as f:
        pickle.dump(greedyIdx_nsp_zero, f)
    output_file=f"nsp_one_greedy_indices_after_step_{completed_steps}.pkl"
    output_file=os.path.join(args.subset_dir, output_file)
    with open(output_file, "wb") as f:
        pickle.dump(greedyIdx_nsp_one, f)


accelerator.wait_for_everyone()
nsp_zero_subset_dataset=nsp_zero.select(subset_indices_nsp_zero[0])
nsp_one_subset_dataset=nsp_one.select(subset_indices_nsp_one[0])
# Concatenate the two datasets
subset_dataset = concatenate_datasets([nsp_zero_subset_dataset, nsp_one_subset_dataset])
subset_dataloader=DataLoader(
    subset_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size)
subset_dataloader = accelerator.prepare(subset_dataloader)

 70%|███████   | 90617/129050 [22:42<09:37, 66.49it/s]

104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104



[||||||||||||||||||||]100% [Iteration 103 of 103]eration 103 of 103]103]Iteration 101 of 103]
[||||||||||||||||||||]100% [Iteration 103 of 103]Iteration 19103% of  [Iteration 20103 of ]103]
[||||||||||||||||||||]100% [Iteration 103 of 103] [Iteration %45 [Iteration  of 101 of 103]103][Iteration 77 of 103]

[|||||||||           ]49% [Iteration 51 of 103]3]3103]]n 48 of 103]103]103]
[||||||||||||||||||||]100% [Iteration 103 of 103]% [Iteration  [Iteration 10179 of  of 103]103]
[||||||||||||||||||||]100% [Iteration 103 of 103]103]
[|                   ]8% [Iteration 9 of 103]




[|||||||             ]38% [Iteration 40 of 103]103]ration ]31 of 103]1% [Iteration 2 of ]103]n 2849 of  of 103103]]103]of ]11103 of ]103]
[|||||||||||||       ]68% [Iterati   of ]67103%] [Iteration 70 of 103]03]]|||||||||||||||64% [Iteration 66 of  ]96% [Iteration 99103 of ]103]|||||||||||||||     ]75% [Iteration 78 of 103] % [Iteration 67 of 103]103]103% of 103103]]
[|||||               ]25% [Iteration 26 of 10

: 

Training the Model

In [ ]:
logger.info("Begin the main training loop with importance re-sampling, after warm-start")
while completed_steps<args.max_train_steps:
    model.train()
    select_subset=False
    for step, batch in enumerate(subset_dataloader):
        train_time=0
        subset_time=0
        start_time=time.time()
        outputs=model(**batch)
        loss=outputs.loss
        logger.info(f"Completed Steps: {1+completed_steps}; Loss: {loss.detach().float()}; lr: {lr_scheduler.get_last_lr()};")
        loss=loss/args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step%args.gradient_accumulation_steps==0 or step==len(subset_dataloader)-1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps+=1
        train_time += (time.time() - start_time)

        if isinstance(checkpointing_steps, int):
            if completed_steps%checkpointing_steps==0:
                output_dir=f"step_{completed_steps }"
                if args.output_dir is not None:
                    output_dir=os.path.join(args.output_dir, output_dir)
                accelerator.save_state(output_dir)

        if completed_steps>=args.max_train_steps:
            break
        
        if (completed_steps)%args.select_every==0:
            select_subset=True
            break
        timing.append([train_time, subset_time])
    if select_subset==True:
        accelerator.wait_for_everyone()
        start_time = time.time()
        num_samples = int(round(len(full_dataset) * args.subset_fraction, 0)) 
        if args.selection_strategy == 'Random-Online':
            if accelerator.is_main_process:
                subset_indices_nsp_zero = [random.sample(list(range(len(first_sent_nsp_zero))), math.floor(num_samples/2))]
                subset_indices_nsp_one = [random.sample(list(range(len(first_sent_nsp_one))), math.ceil(num_samples/2))]
            else:
                subset_indices_nsp_zero = [[]]
                subset_indices_nsp_one = [[]]
        elif args.selection_strategy in ["fl", "logdet", "gc", "disparity-min"]:
            logger.info(f"Performing Subset selection for NSP class 0")
            sampling_start_time=time.time()
            if accelerator.is_main_process:
                subset_indices_nsp_zero=[[]]
                rng=np.random.default_rng(args.seed+completed_steps)
                for i, partition_prob in enumerate(probs_nsp_zero):
                    partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_zero)) * math.floor(num_samples/2)), len(partition_prob)-1)
                    subset_indices_nsp_zero[0].extend(rng.choice(greedyList_nsp_zero[i], size=partition_budget, replace=False, p=partition_prob).tolist())
            else:
                subset_indices_nsp_zero=[[]]
            logger.info("Sampling time(NSP Class 0): {}".format(time.time()-sampling_start_time))
        
            logger.info(f"Performing Subset selection for NSP class 1")
            sampling_start_time=time.time()
            if accelerator.is_main_process:
                subset_indices_nsp_one=[[]]
                rng=np.random.default_rng(args.seed+completed_steps)
                for i, partition_prob in enumerate(probs_nsp_one):
                    partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_one)) * math.ceil(num_samples/2)), len(partition_prob)-1)
                    subset_indices_nsp_one[0].extend(rng.choice(greedyList_nsp_one[i], size=partition_budget, replace=False, p=partition_prob).tolist())
            else:
                subset_indices_nsp_one=[[]]
            logger.info("Sampling time(NSP Class 1): {}".format(time.time()-sampling_start_time))
        accelerator.wait_for_everyone()
        broadcast_object_list(subset_indices_nsp_zero)
        broadcast_object_list(subset_indices_nsp_one)
        timing.append([0, time.time()-start_time])
        if accelerator.is_main_process:
            output_file=f"nsp_zero_subset_indices_after_step_{completed_steps}.pt"
            output_file=os.path.join(args.subset_dir, output_file)
            torch.save(torch.tensor(subset_indices_nsp_zero), output_file)
            output_file=f"nsp_one_subset_indices_after_step_{completed_steps}.pt"
            output_file=os.path.join(args.subset_dir, output_file)
            torch.save(torch.tensor(subset_indices_nsp_one), output_file)
        accelerator.wait_for_everyone()

        nsp_zero_subset_dataset=nsp_zero.select(subset_indices_nsp_zero[0])
        nsp_one_subset_dataset=nsp_one.select(subset_indices_nsp_one[0])
        # Concatenate the two datasets
        subset_dataset = concatenate_datasets([nsp_zero_subset_dataset, nsp_one_subset_dataset])
        subset_dataloader=DataLoader(
            subset_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size)
        subset_dataloader = accelerator.prepare(subset_dataloader)

        subset_time=time.time()-start_time
        select_subset=False
        timing.append([0, subset_time])
        logger.info(f"Subset selection time(total resampling time): {time.time()-start_time} seconds.")
    model.eval()
    losses=[]
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs=model(**batch)

        loss=outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_eval_batch_size)))

    losses=torch.cat(losses)
    losses=losses[:len(eval_dataset)]
    try:
        perplexity=math.exp(torch.mean(losses))
    except OverflowError:
        perplexity=float("inf")

    logger.info(f"Steps {completed_steps}: perplexity: {perplexity}")

logger.info(f"Timing: {timing}")
logger.info(f"Saving the final model after {completed_steps} steps.")
if args.output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model=accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
    )
    if accelerator.is_main_process:
        tokenizer.save_pretrained(args.output_dir)
logger.info(f"Training completed successfully in {time.time()-main_start_time} seconds.")